# **Montando o Drive**

In [1]:
# Coloca os arquivos do google drive disponíveis para serem usados pelo colab

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# **Carregando o Arquivo das Bounding Boxes a ser Usado**

In [18]:
import torch

bboxes_xywh = torch.load('/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/bounding_boxes_xywh.pt', map_location=torch.device('cpu'))

In [17]:
print(bboxes_xywh)

[tensor([[281.0857, 161.4280,  59.6277, 322.0080],
        [376.4540, 156.8315,  56.6536, 283.4524],
        [191.9488, 161.8454,  55.6736, 321.9173],
        [378.5609, 386.7907,  52.7142,  58.4186],
        [ 95.5935, 150.5902,  53.3537, 300.1929],
        [283.1572, 386.4635,  48.5226,  59.0730],
        [ 96.7655, 386.8226,  57.0319,  58.3548],
        [ 16.6350, 240.3738,  32.7101, 345.7437],
        [192.6547, 384.4404,  51.5413,  63.0710],
        [ 17.2421,  24.7319,  34.4842,  49.4577]]), tensor([[ 20.9368, 340.9545,  41.8194, 150.0909],
        [292.8578,  61.1349,  47.2979, 122.1631],
        [ 27.3179,  30.4472,  54.4825,  60.8609],
        [203.5073, 113.6803,  48.7639, 226.7368],
        [294.1582, 287.1207,  46.1957, 210.7330],
        [106.8168, 343.5061,  45.8618, 144.9879],
        [386.9548,  24.0336,  43.7427,  48.0671],
        [386.5199, 121.4454,  46.8834,  98.9813],
        [196.8221, 339.8356,  49.7045, 151.6600]]), tensor([[ 37.9331, 330.9395,  47.4150, 170.09

# **Cálculo das Linhas de Plantação**

In [20]:
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import re

def draw_line_with_length(draw, start, end, fill=(255, 0, 255), width=5, font=None):
    draw.line([start, end], fill=fill, width=width)

folder_path = "/content/drive/MyDrive/TG/arquiteturas/Ultralytics/sorghumfield.v3-416x416_augmented.yolov8/test/images"

image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]

image_paths.sort()

output_folder = '/content/drive/MyDrive/TG/arquiteturas/Ultralytics/YOLOv8/calculos/imagens_linhas'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, (tensor, image_path) in enumerate(zip(bboxes_xywh, image_paths)):
    image = cv2.imread(image_path)
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)

    boxes = tensor.numpy()

    for box in boxes:
        x_center, y_center, width, height = box
        line_start = (x_center, y_center - height / 2)
        line_end = (x_center, y_center + height / 2)
        draw_line_with_length(draw, line_start, line_end)

    image = np.array(pil_image)
    output_image_path = os.path.join(output_folder, f"image_{i}_lines.jpg")
    cv2.imwrite(output_image_path, image)
